In [ ]:
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
import pandas as pd
import scipy.stats as st

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os
import os.path
import argparse
from torch.autograd import Variable
import statsmodels.api as sm

In [ ]:
def round_p(value, round_digits):
    return format(round(value,round_digits), "."+str(round_digits)+"f")
def mse_loss(beta_, c, d, y, t):
    return(c/(1+np.exp(-beta_.dot(t)))+d-y)**2

def avg_y(beta_, c, d, t):
    return c/(1+np.exp(-beta_.dot(t)))+d

def exp(x):
    return np.exp(x)

def generate_y(coef, c, d, x, t, n):
    y = np.zeros(n)
    y_error = 0.05*np.random.uniform(-1, 1, n)
    for i in range(n):
        y[i] = c/(1+np.exp(-(x[i].dot(coef)).dot(t[i]))) + d + y_error[i]
    return y, y_error

def generate_x(d_c, n):
    return np.random.uniform(0, 1, [n, d_c])

def generate_t(t_combo, t_dist, n):
    return t_combo[np.random.choice(np.shape(t_dist)[0], n, p=t_dist),]


def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

def calculate_mse(loader, is_gpu, net):
    """Calculate accuracy.

    Args:
        loader (torch.utils.data.DataLoader): training / test set loader
        is_gpu (bool): whether to run on GPU
    Returns:
        tuple: (overall accuracy, class level accuracy)
    """
    cnt = 0
    total_loss = 0

    for data in loader:
        inputs, labels = data
        if is_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs, labels = Variable(inputs), Variable(labels)
        outputs = net(inputs)
        
        
        cnt += labels.size(0)
        total_loss += sum((outputs-labels)**2)

    return total_loss/float(cnt)

def plot_dev(loader, is_gpu, net):
    cnt = 0
    total_loss = 0
    
    real_y = []
    pred_y = []

    for data in loader:
        inputs, labels = data
        if is_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs, labels = Variable(inputs), Variable(labels)
        outputs = net(inputs)
        
        real_y.append(labels.tolist())
        pred_y.append(outputs.tolist())
        
        cnt += labels.size(0)
        total_loss += sum((outputs-labels)**2)
    real_y = [x for sublist in real_y for x in sublist]
    pred_y = [x for sublist in pred_y for x in sublist]
    print('MSELoss= ', total_loss/float(cnt))
    plt.scatter(real_y, pred_y)
    plt.xlabel('real y')
    plt.ylabel('pred y')
    plt.show()
    

parser = argparse.ArgumentParser()
# hyperparameters settings
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--wd', type=float, default=5e-4, help='weight decay')#lr/(c+wd)
parser.add_argument('--epochs', type=int, default=50,
                    help='number of epochs to train')
parser.add_argument('--batch_size_train', type=int,
                    default=1000, help='training set input batch size')
parser.add_argument('--batch_size_test', type=int,
                    default=1000, help='test set input batch size')
parser.add_argument('--is_gpu', type=bool, default=False,
                    help='whether training using GPU')
import sys
sys.argv=['']
del sys

In [ ]:
n_rep = 5#we replicate 200 times in the paper

# Validation of DeDL

## Data generating process definition

In [ ]:
m = 4  #number of experiments
d_c = 10 #number of features
lr = 0.05

all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]

n_est = int(m*500)
n_train = int(m*500)
reg_term = 0.0005
#reg_loss = 0.001
reg_loss = 0
train_epochs = 2000
test_thres = 0.3
n_cnver_test = 10000

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))

true_0 = []#real ATE in base
true_1 = []#real ATE in treatment

estimator_LR_0 = []#ATE by LR in base
estimator_LR_1 = []#ATE by LR in treatment

estimator_0 = []#ATE by SDL in base
estimator_1 = []#ATE by SDL in treatment

estimator_debias_0 = []#ATE by DeDL in base
estimator_debias_1 = []#ATE by DeDL in treatment

estimator_additive_1 = []#ATE by LA in treatment
mse_c = []
mse_theta = []
mae_theta = []

p_ = []
p_LA = []
p_LR = []
p_DL = []#p value by SDL
p_DDL = []#p value by DeDL
test_err = []

## Comparison of LA, LR, SDL, DeDL

In [ ]:
# FNN+poly with k=4 and 4 exps
class FNN_asig(nn.Module):
    """FNN."""

    def __init__(self):
        """FNN Builder."""
        super(FNN_asig, self).__init__()
        

        self.layer1 = nn.Sequential(
            nn.Linear(d_c, 10, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(10, m+1)
        )
        self.siglayer = nn.Sigmoid()
        self.layer3 = nn.Linear(1, 1, bias=False)


    def forward(self, x):
        """Perform forward."""
        b = self.layer1(x[:,0:d_c])
        u = torch.sum(b*x[:, d_c:], 1)
        u = self.siglayer(u)
        u = u.unsqueeze(1)
        u = self.layer3(u)
        return torch.reshape(u, (-1,))
    
def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

In [ ]:
rep_index = 0

while rep_index < n_rep:
    
    print('# of replication:', rep_index)
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0

    def generate_y_true(coef, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            y[i] = c/(1+np.exp(-((x[i].dot(coef))**3).dot(t[i]))) + d + y_error[i]
        return y, y_error
    def generate_y_true_1(coef, x, t):
        return c_true/(1+np.exp(-((x.dot(coef))**3).dot(t)))
    def generate_beta_true_1(coef, x, t):
        return (x.dot(coef))**3
    
    #generate samples for estimation
    samples_x_est = generate_x(d_c, n_train)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_train)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_train)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_train,1), axis=1)
    data_obs = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
    
    model_LR = sm.OLS(samples_y_est.reshape(n_train,1), np.append(samples_x_est, samples_t_est, axis=1))
    results_LR = model_LR.fit()
    coef_LR = results_LR.params

    opt = parser.parse_args()
    train_samples = int(0.7*data_obs.shape[0])
    x_train_set = np.float32(data_obs)[0:train_samples, 0:-1]
    y_train_set = np.float32(data_obs)[0:train_samples, -1]
    x_test_set = np.float32(data_obs)[train_samples:, 0:-1]
    y_test_set = np.float32(data_obs)[train_samples:, -1]

    trainset = torch.utils.data.TensorDataset(torch.Tensor(x_train_set), torch.Tensor(y_train_set)) # create your datset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=opt.batch_size_train, shuffle=True)

    testset = torch.utils.data.TensorDataset(torch.Tensor(x_test_set), torch.Tensor(y_test_set)) # create your datset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=opt.batch_size_test, shuffle=True)
    
    net = FNN_asig()
    test_accuracy = 100
    print('---------warm-up train---------')
    wp_cnt = 0
    while(test_accuracy >= 1):
        wp_cnt += 1
        net = FNN_asig()
        with torch.no_grad():
            net.layer3.weight[0, 0] = float(np.max(y_train_set))
        criterion = nn.MSELoss()
        #criterion = nn.L1Loss()
        #warm-up train
        test_accuracy = 100
        optimizer = optim.Adam(net.parameters(), lr = 0.1, weight_decay = opt.wd)
        for epoch in range(200):
            for i, data_i in enumerate(trainloader, 0):
                inputs, labels = data_i
                inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = net(inputs)
                l1_norm = sum(p.abs().sum() for p in net.parameters())
                loss = criterion(outputs, labels) + reg_loss*l1_norm
                loss.backward()
                optimizer.step()
            #train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
            test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
            if test_accuracy < 1:
                break
        if wp_cnt >= 5:
            break
    #training 
    print('---------train---------')
    optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay = opt.wd)
    criterion = nn.MSELoss()
    for epoch in range(train_epochs):
        for i, data_i in enumerate(trainloader, 0):
            inputs, labels = data_i
            inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            l1_norm = sum(p.abs().sum() for p in net.parameters())
            loss = criterion(outputs, labels) + reg_loss*l1_norm
            loss.backward()
            optimizer.step()
        train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
        test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
        if epoch%100 == 0:
            print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
                epoch, train_accuracy, test_accuracy))
        if test_accuracy < test_thres and epoch > 500:
            break  
    if test_accuracy > test_thres:
        continue
    else:
        rep_index += 1
        test_err.append(test_accuracy)
    
    activation = {}
    net.layer1.register_forward_hook(get_activation('layer1'))
    
    
    #generate samples for inference
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est,1), axis=1)

    for t_star in t_star_all:
        data_est = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
        #at t=0
        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
        x_all_set=np.float32(data_est)[:, 0:-1]
        y_all_set=np.float32(data_est)[:, -1]
        
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        
        beta_ = []
        pred_y_loss = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            beta_.append(activation['layer1'].tolist())  
            pred_y_loss.append(outputs.tolist())
        beta_ = np.array(beta_).reshape(n_est, m+1)
        pred_y_loss = np.array(pred_y_loss).reshape(n_est)
        
        real_y = samples_y_est.copy()
        
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))

        lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            for i in range(t_combo_obs.shape[0]):
                t = t_combo_obs[i]
                y = avg_y(beta_temp, c_true, d_true, t)
                u = beta_temp.dot(t)
                G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
                lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            try:
                lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
            except:
                print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        estimator_LR_0.append(np.mean(pred_y_LR))#LR
        estimator_0.append(np.mean(pred_y))#SDL
        estimator_debias_0.append(np.mean(pred_y_debiased))#DeDL
        true_0.append(np.mean(real_y_star))#ground truth
        
        #record for comparison - t test
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        pred_y_debiased_0 = pred_y_debiased.copy()
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        for i in range(n_est):
            for j in range(m):
                x_all_set[i][-m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))


        G_theta = []
        
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:
            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))
            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))
            cnt += 1


        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
        
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_1.append(np.mean(real_y_star))
        estimator_LR_1.append(np.mean(pred_y_LR))
        estimator_1.append(np.mean(pred_y))
        estimator_debias_1.append(np.mean(pred_y_debiased))

        p_.append(stats.ttest_ind(real_y_star, real_y_star_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1]) 
        p_DL.append(stats.ttest_ind(pred_y, pred_y_0)[1])
        p_DDL.append(stats.ttest_ind(pred_y_debiased, pred_y_debiased_0)[1])
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))
        
        

In [ ]:
test_err_np = np.zeros(n_rep)
for i in range(n_rep):
    test_err_np[i] = test_err[i].tolist()
test_err =  test_err_np.copy()
del test_err_np

p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_DL = np.array(p_DL).reshape((n_rep, 2**m))
p_DDL = np.array(p_DDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_0 = np.array(estimator_0).reshape((n_rep, 2**m))
estimator_debias_0 = np.array(estimator_debias_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_1 = np.array(estimator_1).reshape((n_rep, 2**m))
estimator_debias_1 = np.array(estimator_debias_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR SDL DeDL
ape_2 = np.zeros([2**m, 4, n_rep])
se = np.zeros([2**m, 4, n_rep])
ae = np.zeros([2**m, 4, n_rep])
CDR = np.zeros([2**m, 4, n_rep])
for i in range(n_rep):
    CDR[0,:,i] = 1
    for j in range(1,2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        dl = estimator_1[i,j]-estimator_0[i,j]
        dedl = estimator_debias_1[i,j]-estimator_debias_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_DL[i,j]<=0.05 and dl*true_eff>0) or (p_[i,j]>0.05 and p_DL[i,j]>0.05):
            CDR[j,2,i] = 1
        if (p_[i,j]<=0.05 and p_DDL[i,j]<=0.05 and dedl*true_eff>0) or (p_[i,j]>0.05 and p_DDL[i,j]>0.05):
            CDR[j,3,i] = 1
            

print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]    
    no_bias_effect = estimator_1[:,index]-estimator_0[:,index]
    debias_effect = estimator_debias_1[:,index]-estimator_debias_0[:,index]
    
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((no_bias_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 3, :] = np.abs((debias_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    ape_2[index, 3, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)    
    se[index, 2, :] = ((no_bias_effect - true_effect)**2)
    se[index, 3, :] = ((debias_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((no_bias_effect - true_effect))
    ae[index, 3, :] = np.abs((debias_effect - true_effect))
    
    index += 1
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of SDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('MSE of DeDL         ','|', np.mean(se[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,3,:], axis=0)), scale=st.sem(np.mean(se[:,3,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of SDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('MAE of DeDL         ','|', np.mean(ae[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,3,:], axis=0)), scale=st.sem(np.mean(ae[:,3,:], axis=0))))
print('------------------------------------')
print('MAPE of LA          ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE of LR          ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE of SDL         ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('MAPE of DeDL        ','|', np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0))))

print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of SDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('CDR of DeDL         ','|', np.mean(CDR[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,3,:], axis=0)), scale=st.sem(np.mean(CDR[:,3,:], axis=0))))

print('------------------------------------')

cnt_bai = np.zeros([n_rep, 4])
for i in range(n_rep):
    max_effect = -10000*np.ones(5)
    max_effect_index = np.zeros(5)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        no_bias_effect = estimator_1[i,index]-estimator_0[i,index]
        debias_effect = estimator_debias_1[i,index]-estimator_debias_0[i,index]
        
        if true_effect > max_effect[4]:
            max_effect[4] = true_effect
            max_effect_index[4] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if no_bias_effect > max_effect[2]:
            max_effect[2] = no_bias_effect
            max_effect_index[2] = index
        if debias_effect > max_effect[3]:
            max_effect[3] = debias_effect
            max_effect_index[3] = index
        
    if max_effect_index[0] == max_effect_index[4]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[4]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[4]:
        cnt_bai[i, 2] = 1
    if max_effect_index[3] == max_effect_index[4]:
        cnt_bai[i, 3] = 1
    
print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of SDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('BAI of DeDL         ','|', np.mean(cnt_bai[:, 3]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 3]), scale=st.sem(cnt_bai[:, 3])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))

## PDL Performance under varying NN size and partially/all observed data

In [ ]:
is_small_dnn = 1#by default for fair comparison
is_all_obs = 0#by default for fair comparison

m = 4  #number of experiments
d_c = 10 #number of features

all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]

is_all_obs = 1
is_small_dnn = 0
n_est = int(m*500)

n_train = int(m*500)
reg_term = 0.0005
#reg_loss = 0.05
reg_loss = 0
train_epochs = 1000
test_thres = 1.0

lr = 0.05

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))



true_0 = []#real ATE in base
true_1 = []#real ATE in treatment

estimator_0 = []#ATE by PDL in base
estimator_1 = []#ATE by PDL in treatment

estimator_LR_0 = []#ATE by LR in base
estimator_LR_1 = []#ATE by LR in treatment
estimator_PDL_0 = []#ATE by PDL in base
estimator_PDL_1 = []#ATE by PDL in treatment

estimator_additive_1 = []#ATE by additive in treatment
p_ = []
p_LA = []
p_LR = []
p_PDL = []

test_err = []

In [ ]:
if is_small_dnn == 0:
    
    class FNN_PDL(nn.Module):
        """FNN."""

        def __init__(self, xavier_init = None):
            """FNN Builder."""
            super(FNN_PDL, self).__init__()


            self.layer1 = nn.Sequential(
                nn.Linear(d_c+m+1, 40, bias=False),
                nn.ReLU(inplace=True),
                nn.Linear(40, 40),
                nn.ReLU(inplace=True),
                nn.Linear(40, 1)
            )

            if (xavier_init is not None):
                for m_ in self.modules():
                    if isinstance(m_, nn.Linear):
                        nn.init.xavier_normal_(m_.weight)



        def forward(self, x):
            """Perform forward."""
            x = self.layer1(x)     
            return torch.reshape(x, (-1,))

else:
    
    class FNN_PDL(nn.Module):
        """FNN."""

        def __init__(self, xavier_init = None):
            """FNN Builder."""
            super(FNN_PDL, self).__init__()


            self.layer1 = nn.Sequential(
                nn.Linear(d_c+m+1, 10, bias=False),
                nn.ReLU(inplace=True),
                nn.Linear(10, 10),
                nn.ReLU(inplace=True),
                nn.Linear(10, 1)
            )

            if (xavier_init is not None):
                for m_ in self.modules():
                    if isinstance(m_, nn.Linear):
                        nn.init.xavier_normal_(m_.weight)



        def forward(self, x):
            """Perform forward."""
            x = self.layer1(x)     
            return torch.reshape(x, (-1,))



In [ ]:
rep_index = 0

while rep_index < n_rep:
    
    print('# of replication:', rep_index)
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0

    def generate_y_true(coef, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            y[i] = c/(1+np.exp(-((x[i].dot(coef))**3).dot(t[i]))) + d + y_error[i]
        return y, y_error
    def generate_y_true_1(coef, x, t):
        return c_true/(1+np.exp(-((x.dot(coef))**3).dot(t)))
    def generate_beta_true_1(coef, x, t):
        return (x.dot(coef))**3
    
    #generate samples for estimation
    samples_x_est = generate_x(d_c, n_train)
    if is_all_obs == 0:
        samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_train)
    else:
        samples_t_est = generate_t(t_combo, t_dist, n_train)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_train)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_train,1), axis=1)
    data_obs = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
    
    model_LR = sm.OLS(samples_y_est.reshape(n_train,1), np.append(samples_x_est, samples_t_est, axis=1))
    results_LR = model_LR.fit()
    coef_LR = results_LR.params
    #results.bse

    opt = parser.parse_args()
    train_samples = int(0.7*data_obs.shape[0])
    x_train_set = np.float32(data_obs)[0:train_samples, 0:-1]
    y_train_set = np.float32(data_obs)[0:train_samples, -1]
    x_test_set = np.float32(data_obs)[train_samples:, 0:-1]
    y_test_set = np.float32(data_obs)[train_samples:, -1]

    trainset = torch.utils.data.TensorDataset(torch.Tensor(x_train_set), torch.Tensor(y_train_set)) # create your datset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=opt.batch_size_train, shuffle=True)

    testset = torch.utils.data.TensorDataset(torch.Tensor(x_test_set), torch.Tensor(y_test_set)) # create your datset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=opt.batch_size_test, shuffle=True)
    
    net = FNN_PDL(xavier_init=1)
    test_accuracy = 100
    print('---------warm-up train---------')
    wp_cnt = 0
    while(test_accuracy >= 1):
        wp_cnt += 1
        net = FNN_PDL()
        criterion = nn.MSELoss()
        test_accuracy = 100
        optimizer = optim.Adam(net.parameters(), lr = 0.1, weight_decay = opt.wd)
        for epoch in range(200):
            for i, data_i in enumerate(trainloader, 0):
                inputs, labels = data_i
                inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = net(inputs)
                l1_norm = sum(p.abs().sum() for p in net.parameters())
                loss = criterion(outputs, labels) + reg_loss*l1_norm
                loss.backward()
                optimizer.step()
            #train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
            test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
            #if epoch%100 == 0:
            #    print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
            #        epoch, train_accuracy, test_accuracy))
            if test_accuracy < 5:
                break
        if wp_cnt >= 5:
            break
    #training 
    print('---------train---------')
    optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay = opt.wd)
    criterion = nn.MSELoss()
    for epoch in range(train_epochs):
        for i, data_i in enumerate(trainloader, 0):
            inputs, labels = data_i
            inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            l1_norm = sum(p.abs().sum() for p in net.parameters())
            loss = criterion(outputs, labels) + reg_loss*l1_norm
            loss.backward()
            optimizer.step()
        train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
        test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
        if epoch%100 == 0:
            print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
                epoch, train_accuracy, test_accuracy))
        if test_accuracy < test_thres and epoch > 200:
            break  
    if test_accuracy > test_thres:
        continue
    else:
        rep_index += 1
        test_err.append(test_accuracy)
    
    
    
    #generate samples for inference
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est,1), axis=1)

    for t_star in t_star_all:
        
        
        data_est = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
        #at t=0
        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
        x_all_set=np.float32(data_est)[:, 0:-1]
        y_all_set=np.float32(data_est)[:, -1]
                
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))
        
        
        true_0.append(np.mean(real_y_star))
        estimator_LR_0.append(np.mean(pred_y_LR))
        estimator_PDL_0.append(np.mean(pred_y))

        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))
        
        
        true_1.append(np.mean(real_y_star))
        estimator_LR_1.append(np.mean(pred_y_LR))
        estimator_PDL_1.append(np.mean(pred_y))
        p_.append(stats.ttest_ind(real_y_star, real_y_star_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1]) 
        p_PDL.append(stats.ttest_ind(pred_y, pred_y_0)[1]) 
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))
        

In [ ]:
test_err_np = np.zeros(n_rep)
for i in range(n_rep):
    test_err_np[i] = test_err[i].tolist()
test_err =  test_err_np.copy()
del test_err_np

p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_PDL = np.array(p_PDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_PDL_0 = np.array(estimator_PDL_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_PDL_1 = np.array(estimator_PDL_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR PDL
ape_2 = np.zeros([2**m, 3, n_rep])
se = np.zeros([2**m, 3, n_rep])
ae = np.zeros([2**m, 3, n_rep])
CDR = np.zeros([2**m, 3, n_rep])
for i in range(n_rep):
    for j in range(2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        pdl = estimator_PDL_1[i,j]-estimator_PDL_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_PDL[i,j]<=0.05 and pdl*true_eff>0) or (p_[i,j]>0.05 and p_PDL[i,j]>0.05):
            CDR[j,2,i] = 1
print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]
    PDL_effect = estimator_PDL_1[:,index]-estimator_PDL_0[:,index]
   
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((PDL_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)
    se[index, 2, :] = ((PDL_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((PDL_effect - true_effect))
    
    
    
    index += 1
    
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of PDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of PDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('------------------------------------')
print('MAPE2 of LA         ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE2 of LR         ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE2 of PDL        ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of PDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('------------------------------------')

cnt_bai = np.zeros([n_rep, 3])
for i in range(n_rep):
    max_effect = -10000*np.ones(4)
    max_effect_index = np.zeros(4)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        PDL_effect = estimator_PDL_1[i,index]-estimator_PDL_0[i,index]
                
        if true_effect > max_effect[3]:
            max_effect[3] = true_effect
            max_effect_index[3] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if PDL_effect > max_effect[2]:
            max_effect[2] = PDL_effect
            max_effect_index[2] = index
    if max_effect_index[0] == max_effect_index[3]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[3]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[3]:
        cnt_bai[i, 2] = 1


print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of PDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))


print('----------------observed combos--------------------')

index = 0
index_1 = 0
ape_2 = np.zeros([2**m-np.shape(t_star_unobs)[0], 3, n_rep])
se = np.zeros([2**m-np.shape(t_star_unobs)[0], 3, n_rep])
ae = np.zeros([2**m-np.shape(t_star_unobs)[0], 3, n_rep])


for t_star in t_star_all:
    if list(t_star) not in t_star_unobs.tolist():
        #print('Unobserved treatment effect when t = ', t_star[1:])
    
        true_effect = true_1[:,index]-true_0[:,index]
    
        add_effect = estimator_additive_1[:,index]
        LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]
        PDL_effect = estimator_PDL_1[:,index]-estimator_PDL_0[:,index]

        ape_2[index_1, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
        ape_2[index_1, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
        ape_2[index_1, 2, :] = np.abs((PDL_effect-true_effect))/(np.abs(true_effect))

        #if p_[:,index]>=0.05:#not significant treatment effect
        ape_2[index_1, 0, p_[:,index]>=0.05] = 0
        ape_2[index_1, 1, p_[:,index]>=0.05] = 0
        ape_2[index_1, 2, p_[:,index]>=0.05] = 0

        se[index_1, 0, :] = ((add_effect - true_effect)**2)
        se[index_1, 1, :] = ((LR_effect - true_effect)**2)
        se[index_1, 2, :] = ((PDL_effect - true_effect)**2)

        ae[index_1, 0, :] = np.abs((add_effect - true_effect))
        ae[index_1, 1, :] = np.abs((LR_effect - true_effect))
        ae[index_1, 2, :] = np.abs((PDL_effect - true_effect))
    
    
        index_1 += 1

    index += 1
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of PDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of PDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('------------------------------------')
print('MAPE2 of LA         ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE2 of LR         ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE2 of PDL        ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))






print('----------------unobserved combos--------------------')

index = 0
index_1 = 0
ape_2 = np.zeros([np.shape(t_star_unobs)[0], 3, n_rep])
se = np.zeros([np.shape(t_star_unobs)[0], 3, n_rep])
ae = np.zeros([np.shape(t_star_unobs)[0], 3, n_rep])


for t_star in t_star_all:
    if list(t_star) in t_star_unobs.tolist():
        #print('Unobserved treatment effect when t = ', t_star[1:])
    
        true_effect = true_1[:,index]-true_0[:,index]
    
        add_effect = estimator_additive_1[:,index]
        LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]
        PDL_effect = estimator_PDL_1[:,index]-estimator_PDL_0[:,index]

        ape_2[index_1, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
        ape_2[index_1, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
        ape_2[index_1, 2, :] = np.abs((PDL_effect-true_effect))/(np.abs(true_effect))

        #if p_[:,index]>=0.05:#not significant treatment effect
        ape_2[index_1, 0, p_[:,index]>=0.05] = 0
        ape_2[index_1, 1, p_[:,index]>=0.05] = 0
        ape_2[index_1, 2, p_[:,index]>=0.05] = 0

        se[index_1, 0, :] = ((add_effect - true_effect)**2)
        se[index_1, 1, :] = ((LR_effect - true_effect)**2)
        se[index_1, 2, :] = ((PDL_effect - true_effect)**2)

        ae[index_1, 0, :] = np.abs((add_effect - true_effect))
        ae[index_1, 1, :] = np.abs((LR_effect - true_effect))
        ae[index_1, 2, :] = np.abs((PDL_effect - true_effect))
    
    
        index_1 += 1

    index += 1
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of PDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of PDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('------------------------------------')
print('MAPE2 of LA         ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE2 of LR         ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE2 of PDL        ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))




# Robustness to DNN Convergence

## Data generating process definition
define delta = 0.1, 0.2, 0.3

In [ ]:
err = 0.1#delta coefficient in the paper
m = 4  #number of experiments
d_c = 10 #number of features

#No need to change
all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]

n_est = int(m*500)
n_train = int(m*500)
reg_term = 0.0005

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))

true_0 = []#real ATE in base
estimator_0 = []#ATE by ML in base
estimator_debias_0 = []#ATE by DML in base
true_1 = []#real ATE in treatment
estimator_1 = []#ATE by ML in treatment
estimator_debias_1 = []#ATE by DML in treatment
estimator_additive_1 = []#ATE by additive in treatment
estimator_LR_0 = []#ATE by LR in base
estimator_LR_1 = []#ATE by LR in treatment
p_ = []
p_LA = []
p_LR = []
p_DL = []
p_DDL = []

## Comparison of LA, LR, SDL, DeDL

In [ ]:
for rep_index in range(n_rep):
    print('# of replication:', rep_index)
    #generate coef and biased coef
    
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0
    
    coef_ = coef.copy()
    c_est = c_true*(1+np.random.uniform(-err, err))
    d_est = 0
    

    def generate_y_true(coef, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            y[i] = (c/(1+np.exp(-((x[i].dot(coef))).dot(t[i]))) + d + y_error[i])
        return y, y_error
    def generate_y_true_1(coef, x, t):
        return c_true/(1+np.exp(-((x.dot(coef))).dot(t)))
    
    
    #generate samples for training LR
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y(coef, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est_xt = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est_xt, samples_y_est.reshape(n_est,1), axis=1)
    model_LR = sm.OLS(samples_y_est.reshape(n_est,1), full_data_est_xt)
    results_LR = model_LR.fit()
    coef_LR = results_LR.params
    
    #generate samples for inference
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y(coef, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est,1), axis=1)
    

    for t_star in t_star_all:
        

        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
    
        pred_y_loss = []
        beta_ = []
        for i in range(n_est):
            beta_biased = samples_x_est[i].dot(coef_)*(1+np.random.uniform(-err, err, m+1))#add bias
            beta_.append(beta_biased)  
            pred_y_loss.append(c_est/(1+np.exp(-(beta_biased).dot(samples_t_est[i]))))
        beta_ = np.array(beta_)
        pred_y_loss = np.array(pred_y_loss)
        real_y = samples_y_est.copy()
        pred_y = []
        for i in range(n_est): 
            pred_y.append(c_est/(1+np.exp(-(beta_[i]).dot(t_star_base))))
        pred_y = np.array(pred_y)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))

            
            
        lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            #G_theta.append(c_true*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            #G_theta_loss.append(c_true*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            for i in range(t_combo_obs.shape[0]):
                t = t_combo_obs[i]
                y = avg_y(beta_temp, c_true, d_true, t)
                u = beta_temp.dot(t)
                #G_prime = c_true*np.exp(-u)/(np.exp(-u)+1)**2*t
                G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
                lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            try:
                lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
                #lambda_inv.append(inv(lambda_))
            except:
                print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_0.append(np.mean(real_y_star))
        estimator_LR_0.append(np.mean(pred_y_LR))
        estimator_0.append(np.mean(pred_y))
        estimator_debias_0.append(np.mean(pred_y_debiased))
        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        pred_y_debiased_0 = pred_y_debiased.copy()
        
        
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        pred_y = []
        for i in range(n_est): 
            pred_y.append(c_est/(1+np.exp(-(beta_[i]).dot(t_star_base))))
        pred_y = np.array(pred_y)
        

        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))


        #lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            #for i in range(t_combo_obs.shape[0]):
            #    t = t_combo_obs[i]
            #    y = avg_y(beta_temp, c_true, d_true, t)
            #    u = beta_temp.dot(t)
            #    G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
            #    lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            #try:
            #    lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
            #except:
            #    print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_1.append(np.mean(real_y_star))        
        estimator_LR_1.append(np.mean(pred_y_LR))
        estimator_1.append(np.mean(pred_y))
        estimator_debias_1.append(np.mean(pred_y_debiased))
        
        p_.append(stats.ttest_rel(real_y_star, real_y_star_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1])
        p_DL.append(stats.ttest_rel(pred_y, pred_y_0)[1])
        p_DDL.append(stats.ttest_rel(pred_y_debiased, pred_y_debiased_0)[1])
        
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))

In [ ]:
p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_DL = np.array(p_DL).reshape((n_rep, 2**m))
p_DDL = np.array(p_DDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_0 = np.array(estimator_0).reshape((n_rep, 2**m))
estimator_debias_0 = np.array(estimator_debias_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_1 = np.array(estimator_1).reshape((n_rep, 2**m))
estimator_debias_1 = np.array(estimator_debias_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR SDL DeDL
ape_2 = np.zeros([2**m, 4, n_rep])
se = np.zeros([2**m, 4, n_rep])
ae = np.zeros([2**m, 4, n_rep])
CDR = np.zeros([2**m, 4, n_rep])
for i in range(n_rep):
    CDR[0,:,i] = 1
    for j in range(1,2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        dl = estimator_1[i,j]-estimator_0[i,j]
        dedl = estimator_debias_1[i,j]-estimator_debias_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_DL[i,j]<=0.05 and dl*true_eff>0) or (p_[i,j]>0.05 and p_DL[i,j]>0.05):
            CDR[j,2,i] = 1
        if (p_[i,j]<=0.05 and p_DDL[i,j]<=0.05 and dedl*true_eff>0) or (p_[i,j]>0.05 and p_DDL[i,j]>0.05):
            CDR[j,3,i] = 1
            

print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]    
    no_bias_effect = estimator_1[:,index]-estimator_0[:,index]
    debias_effect = estimator_debias_1[:,index]-estimator_debias_0[:,index]
    
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((no_bias_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 3, :] = np.abs((debias_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    ape_2[index, 3, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)    
    se[index, 2, :] = ((no_bias_effect - true_effect)**2)
    se[index, 3, :] = ((debias_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((no_bias_effect - true_effect))
    ae[index, 3, :] = np.abs((debias_effect - true_effect))
    
    index += 1
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of SDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('MSE of DeDL         ','|', np.mean(se[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,3,:], axis=0)), scale=st.sem(np.mean(se[:,3,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of SDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('MAE of DeDL         ','|', np.mean(ae[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,3,:], axis=0)), scale=st.sem(np.mean(ae[:,3,:], axis=0))))
print('------------------------------------')
print('MAPE of LA          ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE of LR          ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE of SDL         ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('MAPE of DeDL        ','|', np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0))))

print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of SDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('CDR of DeDL         ','|', np.mean(CDR[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,3,:], axis=0)), scale=st.sem(np.mean(CDR[:,3,:], axis=0))))

print('------------------------------------')

cnt_bai = np.zeros([n_rep, 4])
for i in range(n_rep):
    max_effect = -10000*np.ones(5)
    max_effect_index = np.zeros(5)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        no_bias_effect = estimator_1[i,index]-estimator_0[i,index]
        debias_effect = estimator_debias_1[i,index]-estimator_debias_0[i,index]
        
        if true_effect > max_effect[4]:
            max_effect[4] = true_effect
            max_effect_index[4] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if no_bias_effect > max_effect[2]:
            max_effect[2] = no_bias_effect
            max_effect_index[2] = index
        if debias_effect > max_effect[3]:
            max_effect[3] = debias_effect
            max_effect_index[3] = index
        
    if max_effect_index[0] == max_effect_index[4]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[4]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[4]:
        cnt_bai[i, 2] = 1
    if max_effect_index[3] == max_effect_index[4]:
        cnt_bai[i, 3] = 1
    
print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of SDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('BAI of DeDL         ','|', np.mean(cnt_bai[:, 3]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 3]), scale=st.sem(cnt_bai[:, 3])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))

# Link Function Misspecification

In [ ]:
y_linear = []
y_sigmoid = []
n_cnver_test = 10000
gamma_ = 1

rep_index = 0
while rep_index < 100:
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    coef_linear = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0
    
    def generate_y_true_linear(coef, c, d, x, t, n):
        y = np.zeros(n)
        for i in range(n):
            y[i] = gamma_*(x[i].dot(coef_linear)).dot(t[i])  
        return y
    
    def generate_y_true_sigmoid(coef, c, d, x, t, n):
        y = np.zeros(n)
        for i in range(n):
            y[i] =  c/(1+np.exp(-((x[i].dot(coef))).dot(t[i])))
        return y
                     
    samples_x_cnver = generate_x(d_c, n_cnver_test)
    samples_t_cnver = generate_t(t_combo, t_dist, n_cnver_test)
    y_linear.append(generate_y_true_linear(coef, c_true, d_true, samples_x_cnver, samples_t_cnver, n_cnver_test))
    y_sigmoid.append(generate_y_true_sigmoid(coef, c_true, d_true, samples_x_cnver, samples_t_cnver, n_cnver_test))
    rep_index+=1
    #print(rep_index)
y_linear = np.array(y_linear).reshape(-1)
y_sigmoid = np.array(y_sigmoid).reshape(-1)    
#plt.hist(y_linear, label='linear', density=True, alpha=0.6,bins=20)
#plt.hist(y_sigmoid, label='sigmoid', density=True, alpha=0.6,bins=20)
#plt.xlabel('HTE')
#plt.xlim(-20, 20)
#plt.ylabel('density')
#plt.legend()
#plt.show()

#print('HTE Linear     ','|', np.mean(y_linear),'|',st.t.interval(0.95, len(y_linear)-1, loc=np.mean(y_linear), scale=st.sem(y_linear)))
#print('HTE Sigmoid    ','|', np.mean(y_sigmoid),'|',st.t.interval(0.95, len(y_sigmoid)-1, loc=np.mean(y_sigmoid), scale=st.sem(y_sigmoid)))
from IPython.core.pylabtools import figsize
from matplotlib.ticker import FixedFormatter
figsize(6, 4)
fig, ax = plt.subplots(1, 1)
plt.rcParams['savefig.dpi'] = 1200
plt.rcParams['figure.dpi'] = 1200
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rc('font',family='Times New Roman')
y_linear_1 = y_linear + y_sigmoid
y_linear_2 = 3*y_linear+y_sigmoid
y_linear_5 = 5*y_linear+y_sigmoid
#y_linear_1 = y_linear 
#y_linear_2 = 2*y_linear
#y_linear_5 = 5*y_linear
sns.set_theme(style="white",font="Times New Roman")

sns.kdeplot(y_sigmoid, label='\u03B3 = 0',fill=True,bw_adjust=5)
sns.kdeplot(y_linear_1[np.where((y_linear_1 > -10) & (y_linear_1 < 30))],color="r", label='\u03B3 = 1',fill=True,bw_adjust=5)
sns.kdeplot(y_linear_2[np.where((y_linear_2 > -10) & (y_linear_2 < 30))],color="g", label='\u03B3 = 3',fill=True,bw_adjust=5)
sns.kdeplot(y_linear_5[np.where((y_linear_5 > -10) & (y_linear_5 < 30))],color="orange", label='\u03B3 = 5',fill=True,bw_adjust=5)
plt.xlabel('Outcome Y')
plt.xlim(-10, 30)
plt.ylabel('density')
plt.legend(loc="upper left")
#plt.savefig('misspecific_HTE.png', dpi=600, bbox_inches='tight',pad_inches=0.0) 


## Data generating process definition
define the level of gamma = 0, 1, 3, 5

In [ ]:
gamma_ = 1
m = 4  #number of experiments
d_c = 10 #number of features

all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]


n_est = int(m*500)
n_train = int(m*500)
reg_term = 0.0005
#reg_loss = 0.001
reg_loss = 0
train_epochs = 2000
test_thres = 0.2*gamma_+0.1
if gamma_ == 0:
    test_thres = 0.03
    
n_cnver_test = 10000
lr = 0.05

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))

true_0 = []#real ATE in base
estimator_LR_0 = []#ATE by LR in base
estimator_0 = []#ATE by SDL in base
estimator_debias_0 = []#ATE by DEDL in base

true_1 = []#real ATE in treatment
estimator_LR_1 = []#ATE by LR in treatment
estimator_1 = []#ATE by SDL in treatment
estimator_debias_1 = []#ATE by DEDL in treatment

estimator_additive_1 = []#ATE by additive in treatment
p_ = []
p_LA = []
p_LR = []
p_DL = []
p_DDL = []
test_err = []

## Comparison of LA, LR, SDL, DeDL

In [ ]:
rep_index = 0

while rep_index < n_rep:
    print('# of replication:', rep_index)
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    coef_linear = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0
    

    def generate_y_true(coef, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            y[i] = gamma_*(x[i].dot(coef_linear)).dot(t[i]) + (c/(1+np.exp(-((x[i].dot(coef))).dot(t[i]))) + d + y_error[i])
        return y, y_error
    def generate_y_true_1(coef, x, t):
        return gamma_*(x.dot(coef_linear)).dot(t) + c_true/(1+np.exp(-((x.dot(coef))).dot(t)))

    
    #generate samples for estimation
    samples_x_est = generate_x(d_c, n_train)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_train)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_train)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_train,1), axis=1)
    data_obs = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])

    model_LR = sm.OLS(samples_y_est.reshape(n_train,1), np.append(samples_x_est, samples_t_est, axis=1))
    results_LR = model_LR.fit()
    coef_LR = results_LR.params
    
    opt = parser.parse_args()
    train_samples = int(0.7*data_obs.shape[0])
    x_train_set = np.float32(data_obs)[0:train_samples, 0:-1]
    y_train_set = np.float32(data_obs)[0:train_samples, -1]
    x_test_set = np.float32(data_obs)[train_samples:, 0:-1]
    y_test_set = np.float32(data_obs)[train_samples:, -1]

    trainset = torch.utils.data.TensorDataset(torch.Tensor(x_train_set), torch.Tensor(y_train_set)) # create your datset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=opt.batch_size_train, shuffle=True)

    testset = torch.utils.data.TensorDataset(torch.Tensor(x_test_set), torch.Tensor(y_test_set)) # create your datset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=opt.batch_size_test, shuffle=True)
    net = FNN_asig()
    test_accuracy = 100
    print('---------warm-up train---------')
    wp_cnt = 0
    while(test_accuracy >= 1):
        wp_cnt += 1
        net = FNN_asig()
        with torch.no_grad():
            net.layer3.weight[0, 0] = float(np.max(y_train_set))
        criterion = nn.MSELoss()
        #criterion = nn.L1Loss()
        #warm-up train
        test_accuracy = 100
        optimizer = optim.Adam(net.parameters(), lr = 0.1, weight_decay = opt.wd)
        for epoch in range(200):
            for i, data_i in enumerate(trainloader, 0):
                inputs, labels = data_i
                inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = net(inputs)
                l1_norm = sum(p.abs().sum() for p in net.parameters())
                loss = criterion(outputs, labels) + reg_loss*l1_norm
                loss.backward()
                optimizer.step()
            #train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
            test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
            #if epoch%100 == 0:
            #    print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
            #        epoch, train_accuracy, test_accuracy))
            if test_accuracy < 1:
                break
        if wp_cnt >= 5:
            break
    #training 
    print('---------train---------')
    optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay = opt.wd)
    criterion = nn.MSELoss()
    for epoch in range(train_epochs):
        for i, data_i in enumerate(trainloader, 0):
            inputs, labels = data_i
            inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            l1_norm = sum(p.abs().sum() for p in net.parameters())
            loss = criterion(outputs, labels) + reg_loss*l1_norm
            loss.backward()
            optimizer.step()
        train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
        test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
        if epoch%100 == 0:
            print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
                epoch, train_accuracy, test_accuracy))
        if test_accuracy < test_thres and epoch > 500:
            break  
    if test_accuracy > test_thres:
        continue
    else:
        rep_index += 1
        test_err.append(test_accuracy)
    #rep_index += 1
    
    activation = {}
    net.layer1.register_forward_hook(get_activation('layer1'))

    
    #generate samples for inference
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est,1), axis=1)

    for t_star in t_star_all:
        data_est = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
        #at t=0
        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
        x_all_set=np.float32(data_est)[:, 0:-1]
        y_all_set=np.float32(data_est)[:, -1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        
        beta_ = []
        pred_y_loss = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            beta_.append(activation['layer1'].tolist())  
            pred_y_loss.append(outputs.tolist())
        beta_ = np.array(beta_).reshape(n_est, m+1)
        pred_y_loss = np.array(pred_y_loss).reshape(n_est)
        
        real_y = samples_y_est.copy()
        
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))

        lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            for i in range(t_combo_obs.shape[0]):
                t = t_combo_obs[i]
                y = avg_y(beta_temp, c_true, d_true, t)
                u = beta_temp.dot(t)
                G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
                lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            try:
                lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
            except:
                print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_0.append(np.mean(real_y_star))
        estimator_LR_0.append(np.mean(pred_y_LR))
        estimator_0.append(np.mean(pred_y))
        estimator_debias_0.append(np.mean(pred_y_debiased))
        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        pred_y_debiased_0 = pred_y_debiased.copy()
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        for i in range(n_est):
            for j in range(m):
                x_all_set[i][-m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        

        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))


        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1


        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
        
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_1.append(np.mean(real_y_star))
        estimator_LR_1.append(np.mean(pred_y_LR))
        estimator_1.append(np.mean(pred_y))
        estimator_debias_1.append(np.mean(pred_y_debiased))
        
        p_.append(stats.ttest_ind(real_y_star, real_y_star_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1]) 
        p_DL.append(stats.ttest_ind(pred_y, pred_y_0)[1])
        p_DDL.append(stats.ttest_ind(pred_y_debiased, pred_y_debiased_0)[1])
        
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))

    

In [ ]:
test_err_np = np.zeros(n_rep)
for i in range(n_rep):
    test_err_np[i] = test_err[i].tolist()
test_err =  test_err_np.copy()
del test_err_np

p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_DL = np.array(p_DL).reshape((n_rep, 2**m))
p_DDL = np.array(p_DDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_0 = np.array(estimator_0).reshape((n_rep, 2**m))
estimator_debias_0 = np.array(estimator_debias_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_1 = np.array(estimator_1).reshape((n_rep, 2**m))
estimator_debias_1 = np.array(estimator_debias_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR SDL DeDL
ape_2 = np.zeros([2**m, 4, n_rep])
se = np.zeros([2**m, 4, n_rep])
ae = np.zeros([2**m, 4, n_rep])
CDR = np.zeros([2**m, 4, n_rep])
for i in range(n_rep):
    CDR[0,:,i] = 1
    for j in range(1,2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        dl = estimator_1[i,j]-estimator_0[i,j]
        dedl = estimator_debias_1[i,j]-estimator_debias_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_DL[i,j]<=0.05 and dl*true_eff>0) or (p_[i,j]>0.05 and p_DL[i,j]>0.05):
            CDR[j,2,i] = 1
        if (p_[i,j]<=0.05 and p_DDL[i,j]<=0.05 and dedl*true_eff>0) or (p_[i,j]>0.05 and p_DDL[i,j]>0.05):
            CDR[j,3,i] = 1
            

print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]    
    no_bias_effect = estimator_1[:,index]-estimator_0[:,index]
    debias_effect = estimator_debias_1[:,index]-estimator_debias_0[:,index]
    
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((no_bias_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 3, :] = np.abs((debias_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    ape_2[index, 3, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)    
    se[index, 2, :] = ((no_bias_effect - true_effect)**2)
    se[index, 3, :] = ((debias_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((no_bias_effect - true_effect))
    ae[index, 3, :] = np.abs((debias_effect - true_effect))
    
    index += 1
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of SDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('MSE of DeDL         ','|', np.mean(se[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,3,:], axis=0)), scale=st.sem(np.mean(se[:,3,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of SDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('MAE of DeDL         ','|', np.mean(ae[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,3,:], axis=0)), scale=st.sem(np.mean(ae[:,3,:], axis=0))))
print('------------------------------------')
print('MAPE of LA          ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE of LR          ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE of SDL         ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('MAPE of DeDL        ','|', np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0))))

print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of SDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('CDR of DeDL         ','|', np.mean(CDR[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,3,:], axis=0)), scale=st.sem(np.mean(CDR[:,3,:], axis=0))))

print('------------------------------------')

cnt_bai = np.zeros([n_rep, 4])
for i in range(n_rep):
    max_effect = -10000*np.ones(5)
    max_effect_index = np.zeros(5)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        no_bias_effect = estimator_1[i,index]-estimator_0[i,index]
        debias_effect = estimator_debias_1[i,index]-estimator_debias_0[i,index]
        
        if true_effect > max_effect[4]:
            max_effect[4] = true_effect
            max_effect_index[4] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if no_bias_effect > max_effect[2]:
            max_effect[2] = no_bias_effect
            max_effect_index[2] = index
        if debias_effect > max_effect[3]:
            max_effect[3] = debias_effect
            max_effect_index[3] = index
        
    if max_effect_index[0] == max_effect_index[4]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[4]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[4]:
        cnt_bai[i, 2] = 1
    if max_effect_index[3] == max_effect_index[4]:
        cnt_bai[i, 3] = 1
    
print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of SDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('BAI of DeDL         ','|', np.mean(cnt_bai[:, 3]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 3]), scale=st.sem(cnt_bai[:, 3])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))

## Model Misspecified - High Order Interactions

In [ ]:
gamma_ = 1
m = 4  #number of experiments
d_c = 10 #number of features

all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]


n_est = int(m*500)
n_train = int(m*500)
reg_term = 0.0005
#reg_loss = 0.001
reg_loss = 0
train_epochs = 2000
test_thres = 0.2*gamma_+0.1
if gamma_ == 0:
    test_thres = 0.03
    
n_cnver_test = 10000
lr = 0.05

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))

true_0 = []#real ATE in base
estimator_LR_0 = []#ATE by LR in base
estimator_0 = []#ATE by SDL in base
estimator_debias_0 = []#ATE by DEDL in base

true_1 = []#real ATE in treatment
estimator_LR_1 = []#ATE by LR in treatment
estimator_1 = []#ATE by SDL in treatment
estimator_debias_1 = []#ATE by DEDL in treatment

estimator_additive_1 = []#ATE by additive in treatment
p_ = []
p_LA = []
p_LR = []
p_DL = []
p_DDL = []
test_err = []

In [ ]:
rep_index = 0

while rep_index < n_rep:
    print('# of replication:', rep_index)
    coef_linear = np.random.uniform(0, 0.5, [d_c, 2**m-11])
    c_true = np.random.uniform(10, 20)
    d_true = 0
    

    def generate_y_true(coef_linear, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            t_ = t[i]
            t_ext = np.append(t_, np.array([t_[1]*t_[2], t_[1]*t_[3], t_[1]*t_[4], t_[2]*t_[3], t_[2]*t_[4], t_[3]*t_[4],
                       t_[1]*t_[2]*t_[3], t_[2]*t_[3]*t_[4], t_[3]*t_[4]*t_[1], t_[4]*t_[1]*t_[2], 
                       t_[1]*t_[2]*t_[3]*t_[4]]))
            y[i] = gamma_*(x[i].dot(coef_linear)).dot(t_ext[0:-11]) + y_error[i]
        return y, y_error
    def generate_y_true_1(coef_linear, x, t_):
        t_ext = np.append(t_, np.array([t_[1]*t_[2], t_[1]*t_[3], t_[1]*t_[4], t_[2]*t_[3], t_[2]*t_[4], t_[3]*t_[4],
                       t_[1]*t_[2]*t_[3], t_[2]*t_[3]*t_[4], t_[3]*t_[4]*t_[1], t_[4]*t_[1]*t_[2], 
                       t_[1]*t_[2]*t_[3]*t_[4]]))
        return gamma_*(x.dot(coef_linear)).dot(t_ext[0:-11])

    
    #generate samples for estimation
    samples_x_est = generate_x(d_c, n_train)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_train)
    samples_y_est, samples_y_error_est = generate_y_true(coef_linear, c_true, d_true, samples_x_est, samples_t_est, n_train)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_train,1), axis=1)
    data_obs = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])

    model_LR = sm.OLS(samples_y_est.reshape(n_train,1), np.append(samples_x_est, samples_t_est, axis=1))
    results_LR = model_LR.fit()
    coef_LR = results_LR.params
    
    opt = parser.parse_args()
    train_samples = int(0.7*data_obs.shape[0])
    x_train_set = np.float32(data_obs)[0:train_samples, 0:-1]
    y_train_set = np.float32(data_obs)[0:train_samples, -1]
    x_test_set = np.float32(data_obs)[train_samples:, 0:-1]
    y_test_set = np.float32(data_obs)[train_samples:, -1]

    trainset = torch.utils.data.TensorDataset(torch.Tensor(x_train_set), torch.Tensor(y_train_set)) # create your datset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=opt.batch_size_train, shuffle=True)

    testset = torch.utils.data.TensorDataset(torch.Tensor(x_test_set), torch.Tensor(y_test_set)) # create your datset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=opt.batch_size_test, shuffle=True)
    net = FNN_asig()
    test_accuracy = 100
    print('---------warm-up train---------')
    wp_cnt = 0
    while(test_accuracy >= 1):
        wp_cnt += 1
        net = FNN_asig()
        with torch.no_grad():
            net.layer3.weight[0, 0] = float(np.max(y_train_set))
        criterion = nn.MSELoss()
        #criterion = nn.L1Loss()
        #warm-up train
        test_accuracy = 100
        optimizer = optim.Adam(net.parameters(), lr = 0.1, weight_decay = opt.wd)
        for epoch in range(200):
            for i, data_i in enumerate(trainloader, 0):
                inputs, labels = data_i
                inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = net(inputs)
                l1_norm = sum(p.abs().sum() for p in net.parameters())
                loss = criterion(outputs, labels) + reg_loss*l1_norm
                loss.backward()
                optimizer.step()
            #train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
            test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
            #if epoch%100 == 0:
            #    print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
            #        epoch, train_accuracy, test_accuracy))
            if test_accuracy < 1:
                break
        if wp_cnt >= 5:
            break
    #training 
    print('---------train---------')
    optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay = opt.wd)
    criterion = nn.MSELoss()
    for epoch in range(train_epochs):
        for i, data_i in enumerate(trainloader, 0):
            inputs, labels = data_i
            inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            l1_norm = sum(p.abs().sum() for p in net.parameters())
            loss = criterion(outputs, labels) + reg_loss*l1_norm
            loss.backward()
            optimizer.step()
        train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
        test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
        if epoch%100 == 0:
            print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
                epoch, train_accuracy, test_accuracy))
        if test_accuracy < test_thres and epoch > 500:
            break  
    if test_accuracy > test_thres:
        continue
    else:
        rep_index += 1
        test_err.append(test_accuracy)
    #rep_index += 1
    
    activation = {}
    net.layer1.register_forward_hook(get_activation('layer1'))
    true_value=[]
    for name, param in net.named_parameters():
        if param.requires_grad:
            true_value.append(param.data)
    c_est = np.array(true_value[-1].tolist())[0][0]
    d_est = 0

    
    #generate samples for inference
    samples_x_est = generate_x(d_c, n_est)
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est)
    samples_y_est, samples_y_error_est = generate_y_true(coef_linear, c_true, d_true, samples_x_est, samples_t_est, n_est)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est,1), axis=1)

    for t_star in t_star_all:
        data_est = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
        #at t=0
        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
        x_all_set=np.float32(data_est)[:, 0:-1]
        y_all_set=np.float32(data_est)[:, -1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        
        beta_ = []
        pred_y_loss = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            beta_.append(activation['layer1'].tolist())  
            pred_y_loss.append(outputs.tolist())
        beta_ = np.array(beta_).reshape(n_est, m+1)
        pred_y_loss = np.array(pred_y_loss).reshape(n_est)
        
        real_y = samples_y_est.copy()
        
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        
        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef_linear,samples_x_est[i],t_star_base))

        lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            for i in range(t_combo_obs.shape[0]):
                t = t_combo_obs[i]
                y = avg_y(beta_temp, c_true, d_true, t)
                u = beta_temp.dot(t)
                G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
                lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            try:
                lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
            except:
                print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_0.append(np.mean(real_y_star))
        estimator_LR_0.append(np.mean(pred_y_LR))
        estimator_0.append(np.mean(pred_y))
        estimator_debias_0.append(np.mean(pred_y_debiased))
        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        pred_y_debiased_0 = pred_y_debiased.copy()
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        for i in range(n_est):
            for j in range(m):
                x_all_set[i][-m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=1000, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est)
        

        real_y_star = []
        for i in range(n_est):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef_linear,samples_x_est[i],t_star_base))


        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1


        lambda_inv_loss_prime = []
        for i in range(n_est):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
        
        pred_y_LR = []
        for i in range(n_est): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)

        true_1.append(np.mean(real_y_star))
        estimator_LR_1.append(np.mean(pred_y_LR))
        estimator_1.append(np.mean(pred_y))
        estimator_debias_1.append(np.mean(pred_y_debiased))
        
        p_.append(stats.ttest_ind(real_y_star, real_y_star_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1]) 
        p_DL.append(stats.ttest_ind(pred_y, pred_y_0)[1])
        p_DDL.append(stats.ttest_ind(pred_y_debiased, pred_y_debiased_0)[1])
        
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))

    

In [ ]:
test_err_np = np.zeros(n_rep)
for i in range(n_rep):
    test_err_np[i] = test_err[i].tolist()
test_err =  test_err_np.copy()
del test_err_np

p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_DL = np.array(p_DL).reshape((n_rep, 2**m))
p_DDL = np.array(p_DDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_0 = np.array(estimator_0).reshape((n_rep, 2**m))
estimator_debias_0 = np.array(estimator_debias_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_1 = np.array(estimator_1).reshape((n_rep, 2**m))
estimator_debias_1 = np.array(estimator_debias_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR SDL DeDL
ape_2 = np.zeros([2**m, 4, n_rep])
se = np.zeros([2**m, 4, n_rep])
ae = np.zeros([2**m, 4, n_rep])
CDR = np.zeros([2**m, 4, n_rep])
for i in range(n_rep):
    CDR[0,:,i] = 1
    for j in range(1,2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        dl = estimator_1[i,j]-estimator_0[i,j]
        dedl = estimator_debias_1[i,j]-estimator_debias_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_DL[i,j]<=0.05 and dl*true_eff>0) or (p_[i,j]>0.05 and p_DL[i,j]>0.05):
            CDR[j,2,i] = 1
        if (p_[i,j]<=0.05 and p_DDL[i,j]<=0.05 and dedl*true_eff>0) or (p_[i,j]>0.05 and p_DDL[i,j]>0.05):
            CDR[j,3,i] = 1
            

print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]    
    no_bias_effect = estimator_1[:,index]-estimator_0[:,index]
    debias_effect = estimator_debias_1[:,index]-estimator_debias_0[:,index]
    
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((no_bias_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 3, :] = np.abs((debias_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    ape_2[index, 3, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)    
    se[index, 2, :] = ((no_bias_effect - true_effect)**2)
    se[index, 3, :] = ((debias_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((no_bias_effect - true_effect))
    ae[index, 3, :] = np.abs((debias_effect - true_effect))
    
    index += 1
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of SDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('MSE of DeDL         ','|', np.mean(se[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,3,:], axis=0)), scale=st.sem(np.mean(se[:,3,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of SDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('MAE of DeDL         ','|', np.mean(ae[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,3,:], axis=0)), scale=st.sem(np.mean(ae[:,3,:], axis=0))))
print('------------------------------------')
print('MAPE of LA          ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE of LR          ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE of SDL         ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('MAPE of DeDL        ','|', np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0))))

print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of SDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('CDR of DeDL         ','|', np.mean(CDR[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,3,:], axis=0)), scale=st.sem(np.mean(CDR[:,3,:], axis=0))))

print('------------------------------------')

cnt_bai = np.zeros([n_rep, 4])
for i in range(n_rep):
    max_effect = -10000*np.ones(5)
    max_effect_index = np.zeros(5)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        no_bias_effect = estimator_1[i,index]-estimator_0[i,index]
        debias_effect = estimator_debias_1[i,index]-estimator_debias_0[i,index]
        
        if true_effect > max_effect[4]:
            max_effect[4] = true_effect
            max_effect_index[4] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if no_bias_effect > max_effect[2]:
            max_effect[2] = no_bias_effect
            max_effect_index[2] = index
        if debias_effect > max_effect[3]:
            max_effect[3] = debias_effect
            max_effect_index[3] = index
        
    if max_effect_index[0] == max_effect_index[4]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[4]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[4]:
        cnt_bai[i, 2] = 1
    if max_effect_index[3] == max_effect_index[4]:
        cnt_bai[i, 3] = 1
    
print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of SDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('BAI of DeDL         ','|', np.mean(cnt_bai[:, 3]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 3]), scale=st.sem(cnt_bai[:, 3])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))

# Imbalanced Covariates

## Data generating process definition
define the level of lambda = 2, 1, 0.5

In [ ]:
lambda_level = 2
precision_reb = 1/2 #precision of rebalance, precision_reb=1 means no rebalancing


m = 4  #number of experiments
d_c = 10 #number of features

def generate_x_imb(d_c, n):
    x = np.random.uniform(0, 1, [n, d_c])
    x[:,-1] = np.random.exponential(1.0/lambda_level, n)
    return x

def re_x(x_1, prec=1):
    x_1_b = x_1.copy()
    if prec == 1:
        return x_1_b
    else:
        n_bucket = int(1/prec)
        n_samples = x_1_b.shape[0]
        min_n = 10000
        for lb in np.arange(0,1,prec):
            rb = lb + prec
            min_n = min(min_n, (np.greater(x_1_b[:,-1],lb)*np.less(x_1_b[:,-1],rb)).sum())
        x_emp = []
        for lb in np.arange(0,1,prec):
            rb = lb + prec
            x_temp = x_1_b[np.greater(x_1_b[:,-1],lb)*np.less(x_1_b[:,-1],rb),:]
            x_index = np.random.choice(x_temp.shape[0], min_n, replace=False)
            x_emp.append(x_temp[x_index])
        return np.array(x_emp).reshape(min_n*n_bucket, x_1_b.shape[1])


all_combo = list(powerset(list(np.arange(1, m+1))))
t_combo = []
for i in all_combo:
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo.append(t)
t_combo = np.int16(t_combo)
t_dist = (1/2**m)*np.ones(2**m)
t_combo_obs = []
for i in range(m+1):
    t = np.zeros(m+1)
    t[0] = 1
    t[i] = 1
    t_combo_obs.append(t)
t = np.zeros(m+1)
t[[0,1,2]] = 1
t_combo_obs.append(t)
t_combo_obs = np.int16(t_combo_obs)
t_dist_obs = (1/(m+2))*np.ones(m+2)

t_star_all = t_combo.copy()
idx = (t_combo[:,None]!=t_combo_obs).any(-1).all(1)
t_star_unobs = t_combo[idx]


n_est = int(m*500)
n_train = int(m*500)
reg_term = 0.0005
reg_loss = 0
train_epochs = 2000
test_thres = 0.5
lr = 0.05

feature_list = []
for i in range(d_c):
    feature_list.append(str('x')+str(i+1))
t_list = []
for i in range(m+1):
    t_list.append(str('t')+str(i))

true_0 = []#real ATE in base
estimator_0 = []#ATE by ML in base
estimator_debias_0 = []#ATE by DML in base
estimator_LR_0 = []#ATE by LR in base

true_1 = []#real ATE in treatment
estimator_1 = []#ATE by ML in treatment
estimator_debias_1 = []#ATE by DML in treatment
estimator_LR_1 = []#ATE by LR in treatment

estimator_additive_1 = []#ATE by additive in treatment

p_ = []
p_LA = []
p_LR = []
p_DL = []
p_DDL = []
test_err = []

## Comparison of LA, LR, SDL, DeDL

In [ ]:
rep_index = 0

while rep_index < n_rep:
    
    print('# of replication:', rep_index)
    coef = np.random.uniform(-0.5, 0.5, [d_c, m+1])
    c_true = np.random.uniform(10, 20)
    d_true = 0
    
    def generate_y_true(coef, c, d, x, t, n):
        y = np.zeros(n)
        y_error = 0.05*np.random.uniform(-1, 1, n)
        for i in range(n):
            y[i] = c/(1+np.exp(-((x[i].dot(coef))).dot(t[i]))) + d + y_error[i]
        return y, y_error
    def generate_y_true_1(coef, x, t):
        return c_true/(1+np.exp(-((x.dot(coef))).dot(t)))
    def generate_beta_true_1(coef, x, t):
        return (x.dot(coef))
    
    #generate samples for estimation
    samples_x_est = generate_x_imb(d_c, n_train)
    samples_x_est = re_x(samples_x_est, precision_reb)
    n_train_ = samples_x_est.shape[0]
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_train_)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_train_)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_train_,1), axis=1)
    data_obs = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
    
    model_LR = sm.OLS(samples_y_est.reshape(n_train_,1), np.append(samples_x_est, samples_t_est, axis=1))
    results_LR = model_LR.fit()
    coef_LR = results_LR.params

    opt = parser.parse_args()
    train_samples = int(0.7*data_obs.shape[0])
    x_train_set = np.float32(data_obs)[0:train_samples, 0:-1]
    y_train_set = np.float32(data_obs)[0:train_samples, -1]
    x_test_set = np.float32(data_obs)[train_samples:, 0:-1]
    y_test_set = np.float32(data_obs)[train_samples:, -1]

    trainset = torch.utils.data.TensorDataset(torch.Tensor(x_train_set), torch.Tensor(y_train_set)) # create your datset
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=opt.batch_size_train, shuffle=True)

    testset = torch.utils.data.TensorDataset(torch.Tensor(x_test_set), torch.Tensor(y_test_set)) # create your datset
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=opt.batch_size_test, shuffle=True)
    net = FNN_asig()
    test_accuracy = 100
    print('---------warm-up train---------')
    wp_cnt = 0
    while(test_accuracy >= 1):
        wp_cnt += 1
        net = FNN_asig()
        with torch.no_grad():
            net.layer3.weight[0, 0] = float(np.max(y_train_set))
        criterion = nn.MSELoss()
        #criterion = nn.L1Loss()
        #warm-up train
        test_accuracy = 100
        optimizer = optim.Adam(net.parameters(), lr = 0.1, weight_decay = opt.wd)
        for epoch in range(200):
            for i, data_i in enumerate(trainloader, 0):
                inputs, labels = data_i
                inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = net(inputs)
                l1_norm = sum(p.abs().sum() for p in net.parameters())
                loss = criterion(outputs, labels) + reg_loss*l1_norm
                loss.backward()
                optimizer.step()
            #train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
            test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
            #if epoch%100 == 0:
            #    print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
            #        epoch, train_accuracy, test_accuracy))
            if test_accuracy < 1:
                break
        if wp_cnt >= 5:
            break
    #training 
    print('---------train---------')
    optimizer = optim.Adam(net.parameters(), lr = lr, weight_decay = opt.wd)
    criterion = nn.MSELoss()
    for epoch in range(train_epochs):
        for i, data_i in enumerate(trainloader, 0):
            inputs, labels = data_i
            inputs, labels = Variable(inputs), Variable(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            l1_norm = sum(p.abs().sum() for p in net.parameters())
            loss = criterion(outputs, labels) + reg_loss*l1_norm
            loss.backward()
            optimizer.step()
        train_accuracy = calculate_mse(trainloader, opt.is_gpu, net)
        test_accuracy = calculate_mse(testloader, opt.is_gpu, net)
        if epoch%100 == 0:
            print("Iteration: {0} | Training MSE: {1} | Test MSE: {2}".format(
                epoch, train_accuracy, test_accuracy))
        if test_accuracy < test_thres and epoch > 500:
            break  
    if test_accuracy > test_thres:
        continue
    else:
        rep_index += 1
        test_err.append(test_accuracy)
    #rep_index += 1
    
    activation = {}
    net.layer1.register_forward_hook(get_activation('layer1'))
    
    #generate samples for inference
    samples_x_est = generate_x_imb(d_c, n_est)
    samples_x_est = re_x(samples_x_est, precision_reb)
    n_est_ = samples_x_est.shape[0]
    samples_t_est = generate_t(t_combo_obs, t_dist_obs, n_est_)
    samples_y_est, samples_y_error_est = generate_y_true(coef, c_true, d_true, samples_x_est, samples_t_est, n_est_)
    full_data_est = np.append(samples_x_est, samples_t_est, axis=1)
    full_data_est = np.append(full_data_est, samples_y_est.reshape(n_est_,1), axis=1)

    for t_star in t_star_all:
        data_est = pd.DataFrame(full_data_est, columns = feature_list + t_list + ['y'])
        #at t=0
        t_star_base = np.zeros(m+1)
        t_star_base[0] = 1
        x_all_set=np.float32(data_est)[:, 0:-1]
        y_all_set=np.float32(data_est)[:, -1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=n_est_, shuffle=False)
        
        beta_ = []
        pred_y_loss = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            beta_.append(activation['layer1'].tolist())  
            pred_y_loss.append(outputs.tolist())
        beta_ = np.array(beta_).reshape(n_est_, m+1)
        pred_y_loss = np.array(pred_y_loss).reshape(n_est_)
        
        real_y = samples_y_est.copy()
        
        for j in range(m):
            x_all_set[:, -m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=n_est_, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est_)
        
        #real_y_star = []
        #for i in range(n_est):
        #    real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est[i],t_star_base))

        lambda_inv = []
        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1

            for i in range(t_combo_obs.shape[0]):
                t = t_combo_obs[i]
                y = avg_y(beta_temp, c_true, d_true, t)
                u = beta_temp.dot(t)
                G_prime = np.append(c_est*np.exp(-u)/(np.exp(-u)+1)**2*t, [1/(np.exp(-u)+1)])
                lambda_ += t_dist_obs[i]*2*np.outer(G_prime, G_prime)

            try:
                lambda_inv.append(inv(lambda_ + reg_term*np.eye(m+2)))
            except:
                print('Singular matrix')

        lambda_inv_loss_prime = []
        for i in range(n_est_):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est_):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
        
        pred_y_LR = []
        for i in range(n_est_): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)
        
        
        samples_x_est_true = generate_x(d_c, 10000)
        real_y_star = []
        for i in range(10000):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est_true[i],t_star_base))
        
        

        estimator_0.append(np.mean(pred_y))
        estimator_debias_0.append(np.mean(pred_y_debiased))
        true_0.append(np.mean(real_y_star))
        estimator_LR_0.append(np.mean(pred_y_LR))

        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_LR_0 = pred_y_LR.copy()
        pred_y_0 = pred_y.copy()
        
        #record
        real_y_star_0 = real_y_star.copy()
        pred_y_0 = pred_y.copy()
        pred_y_debiased_0 = pred_y_debiased.copy()
        
        #at t=t_star
        t_star_base = t_star.copy()
        
        for i in range(n_est_):
            for j in range(m):
                x_all_set[i][-m+j] = t_star_base[j+1]
        allset = torch.utils.data.TensorDataset(torch.Tensor(x_all_set), torch.Tensor(y_all_set)) # create your datset
        allloader = torch.utils.data.DataLoader(
            allset, batch_size=n_est_, shuffle=False)
        pred_y = []
        for i, data_ in enumerate(allloader, 0):
            inputs, labels = data_
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            pred_y.append(outputs.tolist())
        pred_y = np.array(pred_y).reshape(n_est_)
        

        G_theta = []
        G_theta_loss = []
        t_obs_ = samples_t_est.copy()
        cnt = 0

        for beta_temp in beta_:

            lambda_ = np.zeros([m+2, m+2])#asig use m+2
            u_ = beta_temp.dot(t_star_base)
            G_theta.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*t_star_base, [1/(np.exp(-u_)+1)]))

            u_ = beta_temp.dot(samples_t_est[cnt])
            G_theta_loss.append(np.append(c_est*np.exp(-u_)/(np.exp(-u_)+1)**2*samples_t_est[cnt], [1/(np.exp(-u_)+1)]))

            cnt += 1


        lambda_inv_loss_prime = []
        for i in range(n_est_):
            lambda_inv_loss_prime.append(2*(pred_y_loss[i]-real_y[i])*lambda_inv[i].dot(G_theta_loss[i]))
        lambda_inv_loss_prime = np.array(lambda_inv_loss_prime)

        pred_y_debiased = []
        for i in range(n_est_):
            pred_y_debiased.append(pred_y[i]-G_theta[i].dot(lambda_inv_loss_prime[i]))
            
        pred_y_LR = []
        for i in range(n_est_): 
            pred_y_LR.append(np.append(samples_x_est[i], t_star_base).dot(coef_LR))
        pred_y_LR = np.array(pred_y_LR)
        
       
        samples_x_est_true = generate_x(d_c, 10000)
        real_y_star = []
        for i in range(10000):
            real_y_star.append(0.05*np.random.uniform(-1, 1)+generate_y_true_1(coef,samples_x_est_true[i],t_star_base))
        
        
        estimator_1.append(np.mean(pred_y))
        estimator_debias_1.append(np.mean(pred_y_debiased))
        estimator_LR_1.append(np.mean(pred_y_LR))
        
        true_1.append(np.mean(real_y_star))
        p_.append(stats.ttest_ind(real_y_star, real_y_star_0)[1])
        p_DL.append(stats.ttest_ind(pred_y, pred_y_0)[1])
        p_DDL.append(stats.ttest_ind(pred_y_debiased, pred_y_debiased_0)[1])
        p_LR.append(stats.ttest_ind(pred_y_LR, pred_y_LR_0)[1]) 
        
        #calculate additive
        pred_y_additive = 0
        pred_y_additive_var = 0
        n_LA_samples = 0
        for single_exp_index in np.where(t_star == 1)[0]:
            ttt = np.zeros(m+1)
            ttt[0] = 1
            ttt[single_exp_index] = 1
            pred_y_additive += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
            pred_y_additive_var += samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].var()
            n_LA_samples += (samples_t_est == ttt).sum()
        ttt = np.zeros(m+1)
        ttt[0] = 1
        pred_y_additive -= (np.sum(t_star))*samples_y_est[np.where((samples_t_est == ttt).all(axis=1))].mean()
        estimator_additive_1.append(pred_y_additive)
        t_value = pred_y_additive/np.sqrt(pred_y_additive_var*(np.sum(t_star))/n_LA_samples)
        p_LA.append(stats.t.sf(abs(t_value), df=1))
        

        

In [ ]:
test_err_np = np.zeros(n_rep)
for i in range(n_rep):
    test_err_np[i] = test_err[i].tolist()
test_err =  test_err_np.copy()
del test_err_np

p_ = np.array(p_).reshape((n_rep, 2**m))
p_LA = np.array(p_LA).reshape((n_rep, 2**m))
p_LR = np.array(p_LR).reshape((n_rep, 2**m))
p_DL = np.array(p_DL).reshape((n_rep, 2**m))
p_DDL = np.array(p_DDL).reshape((n_rep, 2**m))
true_0 = np.array(true_0).reshape((n_rep, 2**m))
estimator_LR_0 = np.array(estimator_LR_0).reshape((n_rep, 2**m))
estimator_0 = np.array(estimator_0).reshape((n_rep, 2**m))
estimator_debias_0 = np.array(estimator_debias_0).reshape((n_rep, 2**m))
true_1 = np.array(true_1).reshape((n_rep, 2**m))
estimator_LR_1 = np.array(estimator_LR_1).reshape((n_rep, 2**m))
estimator_1 = np.array(estimator_1).reshape((n_rep, 2**m))
estimator_debias_1 = np.array(estimator_debias_1).reshape((n_rep, 2**m))
estimator_additive_1 = np.array(estimator_additive_1).reshape((n_rep, 2**m))
index = 0

#LA LR SDL DeDL
ape_2 = np.zeros([2**m, 4, n_rep])
se = np.zeros([2**m, 4, n_rep])
ae = np.zeros([2**m, 4, n_rep])
CDR = np.zeros([2**m, 4, n_rep])
for i in range(n_rep):
    CDR[0,:,i] = 1
    for j in range(1,2**m):
        true_eff = true_1[i,j]-true_0[i,j]
        la = estimator_additive_1[i,j]
        lr = estimator_LR_1[i,j]-estimator_LR_0[i,j]
        dl = estimator_1[i,j]-estimator_0[i,j]
        dedl = estimator_debias_1[i,j]-estimator_debias_0[i,j]
        if (p_[i,j]<=0.05 and p_LA[i,j]<=0.05 and la*true_eff>0) or (p_[i,j]>0.05 and p_LA[i,j]>0.05):
            CDR[j,0,i] = 1
        if (p_[i,j]<=0.05 and p_LR[i,j]<=0.05 and lr*true_eff>0) or (p_[i,j]>0.05 and p_LR[i,j]>0.05):
            CDR[j,1,i] = 1
        if (p_[i,j]<=0.05 and p_DL[i,j]<=0.05 and dl*true_eff>0) or (p_[i,j]>0.05 and p_DL[i,j]>0.05):
            CDR[j,2,i] = 1
        if (p_[i,j]<=0.05 and p_DDL[i,j]<=0.05 and dedl*true_eff>0) or (p_[i,j]>0.05 and p_DDL[i,j]>0.05):
            CDR[j,3,i] = 1
            

print('----------------All combos--------------------')
for t_star in t_star_all:
    #print('Treatment effect when t = ', t_star[1:])
    true_effect = true_1[:,index]-true_0[:,index]
    add_effect = estimator_additive_1[:,index]
    LR_effect = estimator_LR_1[:,index]-estimator_LR_0[:,index]    
    no_bias_effect = estimator_1[:,index]-estimator_0[:,index]
    debias_effect = estimator_debias_1[:,index]-estimator_debias_0[:,index]
    
    ape_2[index, 0, :] = np.abs((add_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 1, :] = np.abs((LR_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 2, :] = np.abs((no_bias_effect-true_effect))/(np.abs(true_effect))
    ape_2[index, 3, :] = np.abs((debias_effect-true_effect))/(np.abs(true_effect))
    
    #if p_[:,index]>=0.05:#not significant treatment effect
    ape_2[index, 0, p_[:,index]>=0.05] = 0
    ape_2[index, 1, p_[:,index]>=0.05] = 0
    ape_2[index, 2, p_[:,index]>=0.05] = 0
    ape_2[index, 3, p_[:,index]>=0.05] = 0
    
    se[index, 0, :] = ((add_effect - true_effect)**2)
    se[index, 1, :] = ((LR_effect - true_effect)**2)    
    se[index, 2, :] = ((no_bias_effect - true_effect)**2)
    se[index, 3, :] = ((debias_effect - true_effect)**2)
    
    ae[index, 0, :] = np.abs((add_effect - true_effect))
    ae[index, 1, :] = np.abs((LR_effect - true_effect))
    ae[index, 2, :] = np.abs((no_bias_effect - true_effect))
    ae[index, 3, :] = np.abs((debias_effect - true_effect))
    
    index += 1
    
print('------------------------------------')
print('MSE of LA           ','|', np.mean(se[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,0,:], axis=0)), scale=st.sem(np.mean(se[:,0,:], axis=0))))
print('MSE of LR           ','|', np.mean(se[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,1,:], axis=0)), scale=st.sem(np.mean(se[:,1,:], axis=0))))
print('MSE of SDL          ','|', np.mean(se[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,2,:], axis=0)), scale=st.sem(np.mean(se[:,2,:], axis=0))))
print('MSE of DeDL         ','|', np.mean(se[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(se[:,3,:], axis=0)), scale=st.sem(np.mean(se[:,3,:], axis=0))))
print('------------------------------------')
print('MAE of LA           ','|', np.mean(ae[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,0,:], axis=0)), scale=st.sem(np.mean(ae[:,0,:], axis=0))))
print('MAE of LR           ','|', np.mean(ae[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,1,:], axis=0)), scale=st.sem(np.mean(ae[:,1,:], axis=0))))
print('MAE of SDL          ','|', np.mean(ae[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,2,:], axis=0)), scale=st.sem(np.mean(ae[:,2,:], axis=0))))
print('MAE of DeDL         ','|', np.mean(ae[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ae[:,3,:], axis=0)), scale=st.sem(np.mean(ae[:,3,:], axis=0))))
print('------------------------------------')
print('MAPE of LA          ','|', np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,0,:]*2**m/np.sum(ape_2[:,0,:]!=0, axis=0), axis=0))))
print('MAPE of LR          ','|', np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,1,:]*2**m/np.sum(ape_2[:,1,:]!=0, axis=0), axis=0))))
print('MAPE of SDL         ','|', np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,2,:]*2**m/np.sum(ape_2[:,2,:]!=0, axis=0), axis=0))))
print('MAPE of DeDL        ','|', np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0)), 
                                                                          scale=st.sem(np.mean(ape_2[:,3,:]*2**m/np.sum(ape_2[:,3,:]!=0, axis=0), axis=0))))

print('------------------------------------')
print('CDR of LA           ','|', np.mean(CDR[:,0,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,0,:], axis=0)), scale=st.sem(np.mean(CDR[:,0,:], axis=0))))
print('CDR of LR           ','|', np.mean(CDR[:,1,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,1,:], axis=0)), scale=st.sem(np.mean(CDR[:,1,:], axis=0))))
print('CDR of SDL          ','|', np.mean(CDR[:,2,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,2,:], axis=0)), scale=st.sem(np.mean(CDR[:,2,:], axis=0))))
print('CDR of DeDL         ','|', np.mean(CDR[:,3,:]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(np.mean(CDR[:,3,:], axis=0)), scale=st.sem(np.mean(CDR[:,3,:], axis=0))))

print('------------------------------------')

cnt_bai = np.zeros([n_rep, 4])
for i in range(n_rep):
    max_effect = -10000*np.ones(5)
    max_effect_index = np.zeros(5)
    for index in range(2**m):
        true_effect = true_1[i,index]-true_0[i,index]
        add_effect = estimator_additive_1[i,index]
        LR_effect = estimator_LR_1[i,index]-estimator_LR_0[i,index]
        no_bias_effect = estimator_1[i,index]-estimator_0[i,index]
        debias_effect = estimator_debias_1[i,index]-estimator_debias_0[i,index]
        
        if true_effect > max_effect[4]:
            max_effect[4] = true_effect
            max_effect_index[4] = index
        if add_effect > max_effect[0]:
            max_effect[0] = add_effect
            max_effect_index[0] = index
        if LR_effect > max_effect[1]:
            max_effect[1] = LR_effect
            max_effect_index[1] = index
        if no_bias_effect > max_effect[2]:
            max_effect[2] = no_bias_effect
            max_effect_index[2] = index
        if debias_effect > max_effect[3]:
            max_effect[3] = debias_effect
            max_effect_index[3] = index
        
    if max_effect_index[0] == max_effect_index[4]:
        cnt_bai[i, 0] = 1
    if max_effect_index[1] == max_effect_index[4]:
        cnt_bai[i, 1] = 1
    if max_effect_index[2] == max_effect_index[4]:
        cnt_bai[i, 2] = 1
    if max_effect_index[3] == max_effect_index[4]:
        cnt_bai[i, 3] = 1
    
print('BAI of LA           ','|', np.mean(cnt_bai[:, 0]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 0]), scale=st.sem(cnt_bai[:, 0])))
print('BAI of LR           ','|', np.mean(cnt_bai[:, 1]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 1]), scale=st.sem(cnt_bai[:, 1])))
print('BAI of SDL          ','|', np.mean(cnt_bai[:, 2]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 2]), scale=st.sem(cnt_bai[:, 2])))
print('BAI of DeDL         ','|', np.mean(cnt_bai[:, 3]),'|',st.t.interval(0.95, n_rep-1, loc=np.mean(cnt_bai[:, 3]), scale=st.sem(cnt_bai[:, 3])))
print('------------------------------------')
print('ABS of ATE          ','|', np.mean(np.mean(np.abs(true_1-true_0), axis=1)),'|',st.t.interval(0.95, n_rep-1, 
                                                                                            loc=np.mean(np.mean(np.abs(true_1-true_0), axis=1)), 
                                                                                            scale=st.sem(np.mean(np.abs(true_1-true_0), axis=1))))